<a href="https://colab.research.google.com/github/KanoknatKr/bsc_dpdm24/blob/main/DPDM24_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FINAL
https://drive.google.com/.../1B8S4D8q9-ddowqDezgLfuS00tIF...
สร้าง model เพื่อ ทำนาย Thal_Group โดยใช้ ข้อมูลจาก training data จาก train_set.csv
ทำนายข้อมูลใน X_test แล้วส่งคำตอบในรูปแบบเดียวกับตัวอย่างใน file y_pred_baseline.csv มาทาง google form นี้ https://forms.gle/Q29PWUgy2ckpYVDk8
กำหนดส่งผลวันสุดท้าย 9 มีนาคม 2568

## Immport Data

In [33]:
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.impute import KNNImputer
from sklearn.metrics import classification_report

In [34]:
from google.colab import drive

# Try unmounting the drive first, in case it was already mounted
drive.flush_and_unmount()

# Mount the drive with force_remount=True to ensure a fresh mount
drive.mount('/content/drive', force_remount=True)

# กำหนด path ของไฟล์
train_path = "/content/drive/MyDrive/BSC_DPDM24_Competition/train_set.csv"
test_path = "/content/drive/MyDrive/BSC_DPDM24_Competition/X_test.csv"
pred_path = "/content/drive/MyDrive/BSC_DPDM24_Competition/y_pred_baseline.csv"


# โหลดข้อมูล
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
pred_df = pd.read_csv(pred_path)

Mounted at /content/drive


In [35]:
train_df

,id number,Xn,Hb(Gm%),Hct(%),MCV(fl),A2 (2-3%),HbF (2%),HbA (95-97%),RBC,MCH,MCHC,MCV-RBC-3*HGB,MCV-RBC-5*HGB,|80-MCV|,|27-MCH|,|80-MCV|*|27-MCH|,Thal_Group
0,67d169,25.0,11.1,34.0,78.0,3.2,0.0,55.9,4.358974,25.464706,32.647059,40.341026,18.141026,2.0,1.535294,3.070588,Abnormal_variants
1,67l168,32.0,13.9,42.0,86.0,3.7,0.3,54.8,5.197674,28.859060,33.557047,35.802326,5.802326,6.0,1.859060,11.154362,Abnormal_variants
2,67i267,43.0,10.1,31.0,78.0,2.6,0.3,56.2,3.974359,25.412903,32.580645,43.725641,23.525641,2.0,1.587097,3.174194,Abnormal_variants
3,67e248,33.0,12.9,38.0,76.0,3.0,0.2,61.2,5.000000,25.800000,33.947368,32.300000,6.500000,4.0,1.200000,4.800000,Abnormal_variants
4,67h277,23.0,12.9,40.0,78.0,3.1,0.0,55.6,5.128205,25.155000,32.250000,34.171795,8.371795,2.0,1.845000,3.690000,Abnormal_variants
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4317,66z431,63.0,10.1,31.0,83.0,2.8,1.8,84.2,3.734940,27.041935,32.580645,48.965060,28.765060,3.0,0.041935,0.125806,Normal
4318,66z436,19.0,11.1,34.0,79.0,2.6,0.6,78.0,4.303797,25.791176,32.647059,41.396203,19.196203,1.0,1.208824,1.208824,Normal
4319,66z439,80.0,10.9,33.9,79.0,3.0,0.2,87.2,4.291139,25.401180,32.153392,42.008861,20.208861,1.0,1.598820,1.598820,Normal
4320,66z445,27.0,13.2,41.4,87.0,2.9,0.3,87.6,4.758621,27.739130,31.884058,42.641379,16.241379,7.0,0.739130,5.173913,Normal


In [36]:
test_df

,id number,Xn,Hb(Gm%),Hct(%),MCV(fl),A2 (2-3%),HbF (2%),HbA (95-97%),RBC,MCH,MCHC,MCV-RBC-3*HGB,MCV-RBC-5*HGB,|80-MCV|,|27-MCH|,|80-MCV|*|27-MCH|
0,66z133,38.0,12.4,37.00,78.0,2.7,1.3,75.4,4.743590,26.140541,33.513514,36.056410,11.256410,2.0,0.859459,1.718919
1,66z297,28.0,12.8,39.40,82.0,2.7,0.0,97.3,4.804878,26.639594,32.487310,38.795122,13.195122,2.0,0.360406,0.720812
2,66u491,78.0,10.9,33.90,79.0,3.1,0.0,96.9,4.291139,25.401180,32.153392,42.008861,20.208861,1.0,1.598820,1.598820
3,66x072,65.0,10.9,33.90,79.0,3.0,0.2,87.9,4.291139,25.401180,32.153392,42.008861,20.208861,1.0,1.598820,1.598820
4,67m031,17.0,11.5,36.00,78.0,2.8,0.8,79.6,4.615385,24.916667,31.944444,38.884615,15.884615,2.0,2.083333,4.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1992,67j317,NaN,8.4,26.00,59.0,2.6,0.7,0.3,4.406780,19.061538,32.307692,29.393220,12.593220,21.0,7.938462,166.707692
1993,66a385,NaN,10.1,31.00,78.0,3.3,0.0,55.0,3.974359,25.412903,32.580645,43.725641,23.525641,2.0,1.587097,3.174194
1994,67g355,53.0,10.4,34.00,77.0,3.0,0.2,49.6,4.415584,23.552941,30.588235,41.384416,20.584416,3.0,3.447059,10.341176
1995,66s121,54.0,10.1,31.00,78.0,2.4,0.3,42.0,3.974359,25.412903,32.580645,43.725641,23.525641,2.0,1.587097,3.174194


In [37]:
pred_df

,Thal_Group
0,Normal
1,Normal
2,Normal
3,Normal
4,Normal
...,...
1992,Normal
1993,Normal
1994,Normal
1995,Normal


## Clean Data

In [38]:
print(train_df.isnull().sum())
print(test_df.isnull().sum())

id number              0
Xn                   435
Hb(Gm%)                7
Hct(%)                 7
MCV(fl)                7
A2 (2-3%)            133
HbF (2%)             116
HbA (95-97%)         116
RBC                    7
MCH                    7
MCHC                   7
MCV-RBC-3*HGB          7
MCV-RBC-5*HGB          7
|80-MCV|               7
|27-MCH|               7
|80-MCV|*|27-MCH|      7
Thal_Group             0
dtype: int64
id number              0
Xn                   187
Hb(Gm%)                2
Hct(%)                 2
MCV(fl)                1
A2 (2-3%)             62
HbF (2%)              59
HbA (95-97%)          59
RBC                    2
MCH                    2
MCHC                   2
MCV-RBC-3*HGB          2
MCV-RBC-5*HGB          2
|80-MCV|               1
|27-MCH|               2
|80-MCV|*|27-MCH|      2
dtype: int64


In [39]:
train_df.Thal_Group.unique()

array(['Abnormal_variants', 'Beta_thal_related', 'HbE_related',
       'Alpha_thal_related', 'Complex_genotypes', 'Normal'], dtype=object)

In [40]:
# ลบ id number
train_df = pd.read_csv(train_path)
test_ids = test_df["id number"]  # เก็บ id number ไว้ใช้ตอนบันทึกผล
test_df = test_df.drop(columns=["id number"])
train_df = train_df.drop(columns=["id number"])


# ใช้ KNNImputer กับ Xn ที่หายไปเยอะ(แทนที่ด้วยค่าจากข้อมูลใกล้เคียงที่สุด)
imputer = KNNImputer(n_neighbors=5)  # ใช้ค่าเฉลี่ยจาก 5 ตัวที่ใกล้ที่สุด
train_df["Xn"] = imputer.fit_transform(train_df[["Xn"]])
test_df["Xn"] = imputer.transform(test_df[["Xn"]])

# เก็บ Thal_Group ไว้ก่อน
Thal_Group = train_df["Thal_Group"]

# ใช้ค่าเฉลี่ยเป็นกลุ่ม (Mean by Thal_Group) เหมาะกับข้อมูลที่มีหมวดหมู่ เช่น Thal_Group ใช้วิธีนี้กับทุกคอลัมน์ ยกเว้น Xn ที่หายเยอะเกินไป
train_df = train_df.groupby("Thal_Group").transform(lambda x: x.fillna(x.mean()))


# คำนวณค่าเฉลี่ยจาก train_df
train_means = train_df.mean()

# เติมค่าว่างใน test_df ด้วยค่าเฉลี่ยของ train_df
test_df = test_df.fillna(train_means)

# นำ Thal_Group กลับมาใส่ใน train_df
train_df["Thal_Group"] = Thal_Group

# แปลง Thal_Group เป็นตัวเลข
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_df["Thal_Group"])

# Define X_train
X_train = train_df.drop(columns=['Thal_Group'])

# Define X_test #Defining X_test here before scaling
X_test = test_df

# แบ่งข้อมูล train/test (สำหรับประเมินโมเดล)
X_train_split, X_val, y_train_split, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# **ปรับสเกลข้อมูลสำหรับ Neural Networks**
scaler = StandardScaler()
X_train_split = scaler.fit_transform(X_train_split)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [41]:
print(train_df.isnull().sum())
print(test_df.isnull().sum())

Xn                   0
Hb(Gm%)              0
Hct(%)               0
MCV(fl)              0
A2 (2-3%)            0
HbF (2%)             0
HbA (95-97%)         0
RBC                  0
MCH                  0
MCHC                 0
MCV-RBC-3*HGB        0
MCV-RBC-5*HGB        0
|80-MCV|             0
|27-MCH|             0
|80-MCV|*|27-MCH|    0
Thal_Group           0
dtype: int64
Xn                   0
Hb(Gm%)              0
Hct(%)               0
MCV(fl)              0
A2 (2-3%)            0
HbF (2%)             0
HbA (95-97%)         0
RBC                  0
MCH                  0
MCHC                 0
MCV-RBC-3*HGB        0
MCV-RBC-5*HGB        0
|80-MCV|             0
|27-MCH|             0
|80-MCV|*|27-MCH|    0
dtype: int64


## สร้าง Model 4 ตัวเพื่อทำนายผล

In [42]:
# สร้างโมเดล 4 แบบ
models = {
    "RandomForest": RandomForestClassifier(n_estimators=100, random_state=42),
    "LogisticRegression": LogisticRegression(max_iter=500, random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric="mlogloss", random_state=42),
    "NeuralNetwork": MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=500, random_state=42)
}

# ปิด Warning ของ XGBoost
warnings.filterwarnings("ignore", category=UserWarning, module="xgboost")

# เทรนและเปรียบเทียบผลลัพธ์ของแต่ละโมเดล
best_model = None
best_f1 = 0
results = []

for name, model in models.items():
    model.fit(X_train_split, y_train_split)
    y_pred = model.predict(X_val)

    acc = accuracy_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred, average="weighted")
    recall = recall_score(y_val, y_pred, average="weighted")
    f1 = f1_score(y_val, y_pred, average="weighted")

    results.append([name, acc, precision, recall, f1])

    # ใช้ F1-score เป็นเกณฑ์เลือกโมเดลที่ดีที่สุด
    if f1 > best_f1:
        best_f1 = f1
        best_model = model

# แสดงผลลัพธ์เป็นตารางสวยงาม
results_df = pd.DataFrame(results, columns=["Model", "Accuracy", "Precision", "Recall", "F1-score"])
results_df = results_df.sort_values(by="F1-score", ascending=False).reset_index(drop=True)

In [43]:
print("\n🔹 Model Performance Comparison 🔹")
print(results_df.to_markdown(index=False))


🔹 Model Performance Comparison 🔹
| Model              |   Accuracy |   Precision |   Recall |   F1-score |
|:-------------------|-----------:|------------:|---------:|-----------:|
| XGBoost            |   0.949133 |    0.94925  | 0.949133 |   0.948578 |
| RandomForest       |   0.944509 |    0.944899 | 0.944509 |   0.944567 |
| NeuralNetwork      |   0.921387 |    0.921254 | 0.921387 |   0.921296 |
| LogisticRegression |   0.790751 |    0.781483 | 0.790751 |   0.767263 |


## BEST MODEL
ในการเลือกโมเดลที่ดีที่สุด เราจะพิจารณาจาก F1-score ซึ่งเป็นค่าที่ใช้วัดสมดุลระหว่าง Precision และ Recall เพราะมันสามารถสะท้อนประสิทธิภาพโดยรวมของโมเดลได้ดีกว่าค่าตัวอื่นๆ โดยเฉพาะเมื่อคุณต้องการให้โมเดลมีการทำนายที่ดีทั้งในด้านการระบุผลบวกที่แท้จริง (Recall) และการหลีกเลี่ยงการทำนายผิดเป็นผลบวก (Precision)

In [49]:
# คำนวณหาค่า F1-score ที่ดีที่สุด
best_model_name = results_df.loc[results_df['F1-score'].idxmax()]['Model'] # Get the name of the model instead of the row
best_model = models[best_model_name]  # Get the model object using its name

# แสดงโมเดลที่ดีที่สุด
print(f"\n📈 โมเดลที่ดีที่สุดเมื่อวัดจาก F1-score คือ : {best_model_name} ด้วยค่า F1-score = {results_df.loc[results_df['Model'] == best_model_name, 'F1-score'].values[0]:.6f}")


📈 โมเดลที่ดีที่สุดเมื่อวัดจาก F1-score คือ : XGBoost ด้วยค่า F1-score = 0.948578


## ทำนายผล Thal_grop ใน X_test
1.   **Abnormal_variants** → หมายถึง ฮีโมโกลบินที่ผิดปกติ ซึ่งอาจเกิดจากการกลายพันธุ์ ส่งผลให้โครงสร้างหรือหน้าที่ของฮีโมโกลบินเปลี่ยนไป
2.  **Alpha_thal_related** → เกี่ยวข้องกับ อัลฟาธาลัสซีเมีย (Alpha Thalassemia) ซึ่งเกิดจากความผิดปกติของยีนที่ควบคุมการสร้างสายอัลฟาของฮีโมโกลบิน
3. **Beta_thal_related** → เกี่ยวข้องกับ เบต้าธาลัสซีเมีย (Beta Thalassemia) ซึ่งเกิดจากความผิดปกติของยีนที่ควบคุมการสร้างสายเบต้าของฮีโมโกลบิน
4. **HbE_related** → เกี่ยวข้องกับ ฮีโมโกลบิน E (HbE) ซึ่งเป็นชนิดหนึ่งของฮีโมโกลบินที่พบบ่อยในเอเชียตะวันออกเฉียงใต้ หากมียีน HbE ร่วมกับเบต้าธาลัสซีเมีย อาจทำให้เกิดภาวะโลหิตจางรุนแรง
5. **Normal** → หมายถึง ฮีโมโกลบินปกติ ไม่มีความผิดปกติที่เกี่ยวข้องกับธาลัสซีเมียหรือฮีโมโกลบินผิดปกติ









In [51]:
# เทรนโมเดลที่ดีที่สุดด้วยข้อมูลทั้งหมดและทำนายข้อมูล X_test
best_model.fit(scaler.fit_transform(X_train), y_train)  # ฝึกโมเดลด้วยข้อมูลทั้งหมด
y_test_pred = best_model.predict(X_test)  # ทำนายผลจาก X_test

# แปลงผลการทำนายกลับเป็นชื่อ Thal_group
y_test_pred_labels = label_encoder.inverse_transform(y_test_pred)

# รับข้อมูลที่ไม่ซ้ำกันจาก y_val และ y_pred
# วิธีนี้ช่วยให้คุณพิจารณาเฉพาะคลาสที่มีอยู่ในข้อมูลที่กำลังประเมินเท่านั้น
unique_classes = np.unique(np.concatenate((y_val, y_pred)))

# กรอง label_encoder.classes_ เพื่อรวมเฉพาะคลาสที่เห็นเท่านั้น
target_names = [label_encoder.classes_[i] for i in unique_classes]

# แสดงผลการทำนายพร้อมชื่อ Thal_group
print(classification_report(y_val, y_pred, target_names=target_names))

                    precision    recall  f1-score   support

 Abnormal_variants       1.00      1.00      1.00         3
Alpha_thal_related       0.84      0.83      0.84       201
 Beta_thal_related       0.93      0.93      0.93        96
       HbE_related       1.00      1.00      1.00       230
            Normal       0.91      0.92      0.92       335

          accuracy                           0.92       865
         macro avg       0.94      0.94      0.94       865
      weighted avg       0.92      0.92      0.92       865



In [56]:
# Create a new DataFrame for the desired output
results_df = pd.DataFrame({'Prediction': y_test_pred})

# Map numerical predictions to Thal_group names using the label_mapping
results_df['Thal_Group'] = results_df['Prediction'].map(label_mapping)


# Save the DataFrame to a CSV file
results_df.to_csv('/content/drive/MyDrive/final_Thal_group_predictions.csv', index=False)

print(f"\n เราใช้ {best_model_name} ในการทำนายผลและบันทึกไฟล์นี้ตามตัวอย่างไปยัง final_Thal_group_predictions.csv เรียบร้อยแล้วครับ ♥ ♥ ♥")


 เราใช้ XGBoost ในการทำนายผลและบันทึกไฟล์นี้ตามตัวอย่างไปยัง final_Thal_group_predictions.csv เรียบร้อยแล้วครับ ♥ ♥ ♥
